In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
PATH = '/content/drive/MyDrive/Colab Notebooks/apart'
df=pd.read_csv(PATH + '/dataset.csv')

In [ ]:
df.head()

In [ ]:
X=df.drop('cost',axis =1)
y = df.cost
X.head()

## 다중공선성 주의 항목 

In [ ]:
X.corr()

per_cost_man - Price_per_squar 0.6656



## 연관성이 높아보이는 변수

In [ ]:
df.corr()

1. Price per squar 
2. Exclusive_area
3. per_cost_man
4. top10(시공사)
5. Floor 층수




## 훈련용 검증용 분리


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size = 0.2,
                                                   random_state = 99)

## 모델

In [ ]:
from lightgbm import LGBMRegressor

In [ ]:
lgb = LGBMRegressor(linear_tree = True,
                   boosting_type = 'gbdt',
                   objective = 'regression',
                   n_estimators = 9000,
                   learning_rate = 0.001,
                   max_depth = 4,
                   n_jobs = -1)

lgb.fit(X_train, y_train)

In [ ]:
MSE = mean_squared_error(y_test, lgb.predict(X_test))

In [ ]:
from math import sqrt
print(f'MSE : {MSE}')
print(f'RMSE : {sqrt(MSE)}')

In [ ]:
lgb.feature_importances_

In [ ]:
X.columns

In [ ]:
plt.figure(figsize=(7,5))
sns.barplot(lgb.feature_importances_,['Gu_Label', 'Dong_Label', 'Year', 'Exclusive_area', 'Floor',
       'Price per squar', 'top10', 'YN_r', 'Hospital', 'Station',
       'per_cost_man'])
plt.show()

### 'Hospital', 'Station' 탈락

In [ ]:
df1 = df.drop(['Hospital', 'Station'],axis=1)

In [ ]:
X=df1.drop('cost',axis =1)
y = df1.cost
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size = 0.2,
                                                   random_state = 99)

In [ ]:
lgb = LGBMRegressor(linear_tree = True,
                   boosting_type = 'gbdt',
                   objective = 'regression',
                   n_estimators = 9000,
                   learning_rate = 0.001,
                   max_depth = 4,
                   n_jobs = -1)

lgb.fit(X_train, y_train)

In [ ]:
MSE = mean_squared_error(y_test, lgb.predict(X_test))
print(f'MSE : {MSE}')
print(f'RMSE : {sqrt(MSE)}')

In [ ]:
lgb.feature_importances_

In [ ]:
X.columns

In [ ]:
plt.figure(figsize=(7,5))
sns.barplot(lgb.feature_importances_,['Gu_Label', 'Dong_Label', 'Year', 'Exclusive_area', 'Floor',
       'Price per squar', 'top10', 'YN_r', 'per_cost_man'])
plt.show()

## 3개 추가제거

In [ ]:
df2 = df.drop(['Price per squar','top10', 'YN_r','Hospital', 'Station'],axis=1)

In [ ]:
X=df2.drop('cost',axis =1)
y = df2.cost
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size = 0.2,
                                                   random_state = 99)

In [ ]:
lgb = LGBMRegressor(linear_tree = True,
                   boosting_type = 'gbdt',
                   objective = 'regression',
                   n_estimators = 9000,
                   learning_rate = 0.001,
                   max_depth = 4,
                   n_jobs = -1)

lgb.fit(X_train, y_train)

MSE = mean_squared_error(y_test, lgb.predict(X_test))
print(f'MSE : {MSE}')
print(f'RMSE : {sqrt(MSE)}')

## 하이퍼 파라미터 튜닝

In [ ]:
X=df.drop('cost',axis =1)
y = df.cost
X.head()

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,
                                                 random_state = 99)

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'learning_rate' : [0.1, 0.01, 0.001, 0.0001],
    'max_depth' : [1,2,3,4]
}

grid_cv = GridSearchCV(lgb, param_grid = params, cv=4, scoring='neg_mean_squared_error', verbose=1)
grid_cv.fit(X_train, y_train)

In [ ]:
print(f'최적의 하이퍼 파라미터 : {grid_cv.best_params_}')
print(f'예측 오차 : {grid_cv.best_score_}')

In [ ]:
lgb_r = LGBMRegressor(linear_tree = True,
                   boosting_type = 'gbdt',
                   objective = 'regression',
                   n_estimators = 9000,
                   learning_rate = 0.1,
                   max_depth = 3,
                   n_jobs = -1)

lgb_r.fit(X_train, y_train)

In [ ]:
MSE = mean_squared_error(y_test, lgb_r.predict(X_test))
print(f'MSE : {MSE}')
print(f'RMSE : {sqrt(MSE)}')

In [ ]:
lgb_r.feature_importances_

In [ ]:
X.columns

In [ ]:
lgb_r.feature_importances_.shape

In [ ]:
X.columns.shape

In [ ]:
pred = lgb_r.predict(X_test)

In [ ]:
pred[0]

In [ ]:
y_test.iloc[0]